## Script to 
1. pre-process the raw dose-response files into standard format; 
2. Merge them for all pairs and common pairs; 
3. save the GDSC dosage range for all pairs and drugs

*** Just use CCLE_dose_response.csv and GDSC1_dose_response.csv in data_2019_12_05 fodler as it is gnerated from new metadata file whihc is updated (has 5 dosage drugs and new standard names fro gdsc drugs eg: 681640)

In [1]:
__author__ = 'Aanchal'

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import numpy as np
import FUNCTIONS as f


In [4]:
dataname='GDSC'#'CCLE'#'CCLE'

In [5]:
if (dataname=="GDSC"):
    cl_colname_2b_mapped=dataname+' id'
    dr_colname_2b_mapped=dataname+' id (1)'
    dr_colname_2b_mapped_2=dataname+' id (2)'
    
    drug_col_name='DRUG_ID'#'Compound' #
    cl_col_name='COSMIC_ID'#'CCLE Cell Line Name' #
    dosage_col_name='dose'
    response_col_name='response'
    
    
else:
    cl_colname_2b_mapped=dataname+' name'
    dr_colname_2b_mapped=dataname+' name'
    
    drug_col_name='Compound' #
    cl_col_name='CCLE Cell Line Name' #
    dosage_col_name='Doses (uM)'
    response_col_name='Activity Data (median)'
    

#### read metadata files and raw files

In [6]:
metadata_filename="../data/metadata_2019_08_15_aanchal.xlsx"
cell_line_metadata =pd.read_excel(metadata_filename,sheetname="cell line")
drug_metadata =pd.read_excel(metadata_filename,sheetname="drug")

raw=pd.read_csv("../data/drug_response/"+dataname+"/"+dataname+"_dose_response_scores.tsv", sep="\t")
raw=raw[[cl_col_name,drug_col_name,dosage_col_name,response_col_name]] #224510 pairs, 265 drugs  #len(set(raw[drug_col_name]))

In [8]:
if(dataname=="GDSC"):
    raw=f.remove5dosageDrugs(raw,cl_col_name,drug_col_name,dosage_col_name,response_col_name) #190921 pairs left in GDSC #earlier 265 drugs, now 226 drugs

### 1. Convert to standard format
#### a) map dataset specific names to standard names

In [7]:
[dic_cl,  dic_dr]=f.getDictionary(dataname)

In [8]:
l1=raw[cl_col_name]
l2=raw[drug_col_name]

la=[]
lb=[]

for i in range(len(l1)):
    cl_name_data_specific=l1[i]
    dr_name_data_specific=l2[i]
    
    std_name_cl = dic_cl.get ( str(cl_name_data_specific) )
    std_name_dr = dic_dr.get ( dr_name_data_specific )
    
    la.append(std_name_cl)
    lb.append( std_name_dr)
       
# add these lists as cols to data frame
raw_new=raw.assign(Standard_cell_line_name = la,Standard_drug_name=lb)

#reorder cols
raw_new=raw_new[['Standard_cell_line_name',cl_col_name,'Standard_drug_name',drug_col_name,dosage_col_name,response_col_name]]
###raw_new.to_csv('../data/'+dataname+'_dose_response_no5dosageDrugs.csv',index=False)

#### b) make separate row for every dosage

In [9]:
lens = raw_new[dosage_col_name].str.split(',').map(len)

raw_final=pd.DataFrame({'Standard_cell_line_name': np.repeat(raw_new['Standard_cell_line_name'], lens),
                    cl_col_name: np.repeat(raw_new[cl_col_name], lens),
                    'Standard_drug_name': np.repeat(raw_new['Standard_drug_name'], lens),
                    drug_col_name: np.repeat(raw_new[drug_col_name], lens),
                    dosage_col_name: f.chainer(raw_new[dosage_col_name]),
                    response_col_name: f.chainer(raw_new[response_col_name])})
#rename cols
raw_final.columns = ['Standard cell line name','cell line name','Standard drug name','drug name', 'Doses (uM)','Response' ]

In [10]:
raw_final.to_csv('../data/'+dataname+'_dose_response.csv',index=False)

### Run the above code for both CCLE and GDSC befoe running the below code

### 2. Merge datasets
#### a) All pairs from both datasets

In [11]:
datasets_list=['CCLE','GDSC']
l=[]
for i in range( len (datasets_list) ):
    dataset=pd.read_csv("../data/"+datasets_list[i]+"_dose_response.csv")
    dataset=dataset[['Standard cell line name','Standard drug name','Doses (uM)','Response']]
    l.append(dataset)
data_merged=pd.concat(l) # this is the full merged dataset with all pairs (may not be in common set)

In [12]:
data_merged.to_csv('../data/MERGED_dose_response.csv')
    

#### b) Merge datasets only for common pairs 

In [13]:
import FUNCTIONS as f
datasets_list=['CCLE','GDSC']
rawFileDir="../data/"
[Intersection, Intersection_names] = f.getCommonPairs(datasets_list, rawFileDir)

0
1


In [14]:
Intersection.to_csv('../data/CCLE_GDSC1_dose_response_commonPairs.csv', index=False)

### 3. Save dosage ranges 
##### a)Get min and max dosage tested for each pair (since come CLs dont follow the same range  in a drug in GDSC
eg: drugs 1047 (Nutlin-3a), 1054 (palbo), 1060(PD), 1062 (Selu)

In [5]:
#data_raw=pd.read_csv('../data/GDSC_dose_response.csv')
dataname="GDSC1"#"CCLE" #"CCLE" #
rawFileDir="../data/data_2019_12_05/"

sample_col_name='cell line name'
drug_col_name ='drug name'
dosage_col_name='Dosage (uM)' 
response_col_name='Response'
                                 
data_raw=pd.read_csv(rawFileDir+dataname+"_dose_response.csv")

In [6]:
unique_drugs= data_raw[drug_col_name].unique() 
len(unique_drugs)
drnames=[]
clnames=[]
min_dosages=[]
max_dosages=[]
nod=[]
i=0
for drname in unique_drugs:
    
    i=i+1
    df = data_raw[ data_raw[drug_col_name]==drname] 
    
    unique_cls= df[sample_col_name].unique() 
    print(i, drname,len(unique_cls))
    
    for clname in unique_cls:
        df2 = df[ df[sample_col_name]==clname] 
    
        drnames.append(drname)
        clnames.append(clname)
        min_dosages.append(df2[dosage_col_name].min())
        max_dosages.append(df2[dosage_col_name].max())
        nod.append(len(df2[dosage_col_name].unique()))
        

1 Docetaxel 951
2 VX-702 951
3 Vorinostat 951
4 Gefitinib 951
5 Olaparib 958
6 SN-38 959
7 Nilotinib 951
8 CI-1040 938
9 Veliparib 951
10 Vinblastine 951
11 Lestaurtinib 951
12 Lenalidomide 951
13 Temsirolimus 951
14 Bosutinib 951
15 Cisplatin 951
16 Axitinib 951
17 Elesclomol 951
18 AZD7762 951
19 Cytarabine 951
20 GW441756 951
21 Afatinib 958
22 Methotrexate 951
23 SB216763 857
24 AICA Ribonucleotide 951
25 Vismodegib 951
26 Tretinoin 951
27 SL0101 934
28 PD173074 949
29 ZM447439 949
30 Wee1 Inhibitor 859
31 Nutlin-3a 949
32 Navitoclax 949
33 JNK Inhibitor VIII 949
34 RO-3306 949
35 Motesanib 949
36 Doramapimod 949
37 Tanespimycin 949
38 NU7441 948
39 KU-55933 948
40 BX795 948
41 PLX-4720 956
42 Dactolisib 939
43 Refametinib 954
44 PD0325901 939
45 AZD8055 939
46 Pictilisib 954
47 SB590885 918
48 Selumetinib 954
49 Palbociclib 910
50 MK-2206 910
51 Rapamycin 407
52 WH-4-023 400
53 Tozasertib 400
54 WZ-1-84 400
55 Imatinib 408
56 BI-2536 401
57 NVP-TAE684 407
58 BMS-536924 960
59 Criz

In [7]:
dosage_range_perPair=pd.DataFrame()
dosage_range_perPair[drug_col_name]=drnames
dosage_range_perPair[sample_col_name]=clnames
dosage_range_perPair['min_dosage']=min_dosages
dosage_range_perPair['max_dosage']=max_dosages
dosage_range_perPair['num dosages']=nod
dosage_range_perPair #(its length doesnt match with no of pairs in gdsc..debug y..bcz raw_new has duplicates bcz of two dryug ids for same drug!!!)

,drug name,cell line name,min_dosage,max_dosage,num dosages
0,Docetaxel,K5,0.000049,0.0125,9
1,Docetaxel,D-336MG,0.000049,0.0125,9
2,Docetaxel,SU8686,0.000049,0.0125,9
3,Docetaxel,PC-14,0.000049,0.0125,9
4,Docetaxel,HCC-827,0.000049,0.0125,9
5,Docetaxel,IGR-37,0.000049,0.0125,9
6,Docetaxel,MMAC-SF,0.000049,0.0125,9
7,Docetaxel,MKN45,0.000049,0.0125,9
8,Docetaxel,EBC-1,0.000049,0.0125,9
9,Docetaxel,RERF-GC-1B,0.000049,0.0125,9


In [8]:
dosage_range_perPair.to_csv('../data_processed/'+dataname+'_pair_dosage_range.csv',index=False)

##### b) get min and max dosage tested for each drug (some CLs dont folow the same range); so just take doasge rangeof majority CLs

In [9]:
#dataname='CCLE' #ccle has same dosage range for all pairs so the below code would run only for gdsc
dataname='GDSC1'
dosage_range_perPair=pd.read_csv('../data_processed/'+dataname+'_pair_dosage_range.csv')

In [10]:
unique_drugs= dosage_range_perPair[drug_col_name].unique() 

In [12]:
min_dosages=[]
max_dosages=[]
nod=[]
for drname in unique_drugs:
    df = dosage_range_perPair[ dosage_range_perPair[drug_col_name]==drname]
    
    min_dosages.append(df['min_dosage'].value_counts().argmax())
    max_dosages.append(df['max_dosage'].value_counts().argmax())
    nod.append(df['num dosages'].mean())

c:\python36\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  import sys
c:\python36\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  


In [13]:
dosage_range_perDrug=pd.DataFrame()
dosage_range_perDrug[drug_col_name]=unique_drugs
dosage_range_perDrug['min_dosage']=min_dosages
dosage_range_perDrug['max_dosage']=max_dosages
dosage_range_perDrug['num dosages']=nod
dosage_range_perDrug

,drug name,min_dosage,max_dosage,num dosages
0,Docetaxel,0.000049,0.0125,9.000000
1,VX-702,0.007812,2.0000,9.000000
2,Vorinostat,0.039062,10.0000,9.000000
3,Gefitinib,0.001953,0.5000,9.000000
4,Olaparib,0.019531,10.0000,9.914405
5,SN-38,0.000195,0.1000,9.930136
6,Nilotinib,0.007812,2.0000,9.000000
7,CI-1040,0.039062,10.0000,9.000000
8,Veliparib,0.019531,5.0000,9.000000
9,Vinblastine,0.000391,0.1000,9.000000


In [14]:
dosage_range_perDrug.to_csv('../data_processed/'+dataname+'_drug_dosage_range.csv',index=False)